In [ ]:
(require '[clojupyter.misc.helper :as helper])
(helper/add-dependencies '[metasoarous/oz "1.5.6"])
(helper/add-dependencies '[techascent/tech.ml.dataset "6.033"])
(require '[tech.v3.dataset :as ds])
(require '[oz.notebook.clojupyter :as oz])
(helper/add-dependencies '[clj-time "0.15.2"])
(require '[clj-time.format :as f])




(def cases-per-age (ds/->dataset "https://opendata.ecdc.europa.eu/covid19/agecasesnational/csv/data.csv"))

(def age-groups (set (cases-per-age "age_group")))


(defn parse-date [in]
  (f/parse (f/formatter "MM/dd/YY") in))

(defn get-cases-for-age-group [country age-group]
    ((-> cases-per-age
        (ds/filter-column "country" #(= % country))
        (ds/filter-column "age_group" #(= % age-group)))
        "new_cases"))

(defn year-week [country] 
    ((-> cases-per-age
        (ds/filter-column "country" #(= % country))) 
        "year_week"))

(print (set (cases-per-age "country")))

;; (def values (let [y (get-cases-for-age-group "Romania" "<15yr") t (year-week "Romania")] 
;;                (map #(hash-map :time %1 :item "Romania <15yr" :quantity %2) t y)))

;; (print (year-week "Romania"))




(defn create-oz-values [cntry]
    (flatten (let [t (year-week cntry)](for [age (set (cases-per-age "age_group")) 
          :let [y (get-cases-for-age-group cntry age)]] 
        (map #(hash-map :time %1 :item age :quantity %2) t y)))))


(defn age-graph [cntry]
  {:title  ( str cntry " - Cases by age group - weekly counts")
   :data {:values (create-oz-values cntry)}
   :mark "line"
   :height 400
   :width 800
   :encoding {:x {:field "time"  :type "ordinal" :title "Week number" }
              :y {:field "quantity"  :title "Cases" :scale {:type "pow"}}
              :color {:field "item"}}})




In [ ]:
(oz/view! (age-graph "Romania"))


In [ ]:
(oz/view! (age-graph "Ireland"))

In [ ]:
(oz/view! (age-graph "France"))


In [ ]:
(oz/view! (age-graph "Germany"))

In [ ]:
(oz/view! (age-graph "Spain"))

In [ ]:
(oz/view! (age-graph "Austria"))